In [1]:
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import json  
import networkx as nx
import csv
from multiprocessing import Pool

In [2]:
Yelp_Users = pd.read_json('YelpDataset/user.json',lines=True) # read in user dataset
Yelp_Users['number of friends'] = Yelp_Users['friends'].apply(len) # get number of friends
df_usersNarrow = Yelp_Users[Yelp_Users['number of friends']>0] # sifting dataset to people who have at least one friend
df_usersNarrow = df_usersNarrow[['user_id','friends','number of friends']]

In [3]:
Yelp_ReviewsWithLatCats = pd.read_csv("Yelp_ReviewsWITHLATENTCATEGORIES.csv")

/opt/condaenvs/jupyterhub/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
Yelp_ReviewsWithLatCats = Yelp_ReviewsWithLatCats[['business_id','review_id','user_id','stars','businessLatentCategory']]

In [5]:
Yelp_ReviewsWithLatCats = Yelp_ReviewsWithLatCats.dropna()

In [6]:
reviewsGroupBy = Yelp_ReviewsWithLatCats.groupby("user_id")
listUserIDsReviews = list(reviewsGroupBy.groups.keys())
setOfUserIdsReviews = set(listUserIDsReviews)
#print(len(setOfUserIdsReviews))
df_usersNarrowSet = set(df_usersNarrow['user_id'])
#print(len(df_usersNarrowSet)) ## ONLY WANT REVIEW IN GROUP BY IF IN DF _USERS NARROW!!!!!!

1325415
760008


In [7]:
Yelp_ReviewsWithLatCats = Yelp_ReviewsWithLatCats[Yelp_ReviewsWithLatCats['user_id'].isin(df_usersNarrowSet)]

In [8]:
groupbyUSERandCAT = Yelp_ReviewsWithLatCats.groupby(['user_id','businessLatentCategory'])

In [9]:
REV_ListOfTuplesUSERS_BUSCAT = list(groupbyUSERandCAT.groups) ## LIST OF TUPLES

In [11]:
Yelp_Reviews_WithFriendsGroupBy = Yelp_ReviewsWithLatCats.groupby('user_id')

In [12]:
# Using dictionary comprehension to change Yelp_Reviews_WithFriendsGroupBy to dictionary so we can feed it to 
# the friends network as a dictionary
Yelp_ReviewsUserDict = {g: d for g, d in Yelp_Reviews_WithFriendsGroupBy}

In [13]:
REV_ListOfUSERS = list(Yelp_Reviews_WithFriendsGroupBy.groups)

In [ ]:
## PREPPING ASSORTATIVITY BASED ON STAR AVERAGE

In [47]:
Yelp_ReviewsAggStarMean = Yelp_ReviewsWithLatCats.groupby("user_id").agg("mean")

In [73]:
Yelp_ReviewsAggStarMeanDict = dict(zip(Yelp_ReviewsAggStarMean.index,Yelp_ReviewsAggStarMean.stars))

In [ ]:
## PREPPING ASSORTATIVITY BASED ON REVIEW COUNT

In [90]:
YelpReviewsRevCountSeries = Yelp_ReviewsWithLatCats.groupby("user_id").size()

In [91]:
YelpReviewsRevCountSeriesDict = YelpReviewsRevCountSeries.to_dict()

In [ ]:
## PREPPING ASSORTATIVITY BASED ON CATEGORY AVERAGE RATINGS

In [99]:
busLatCatsList = list(Yelp_ReviewsWithLatCats['businessLatentCategory'].unique())

In [120]:
siftedReviewDfsList = []
siftedListDicts = []
siftedListEnthusDicts = []
for latentCat in busLatCatsList:
    siftedDf = Yelp_ReviewsWithLatCats[Yelp_ReviewsWithLatCats['businessLatentCategory'] == latentCat]
    siftedReviewDfsList.append((latentCat,siftedDf))
    print(latentCat)
    SiftedAggStarMean = siftedDf.groupby("user_id").agg("mean")
    SiftedAggStarMeanDict = dict(zip(SiftedAggStarMean.index,SiftedAggStarMean.stars))
    siftedListDicts.append((latentCat,SiftedAggStarMeanDict))
    
    
    SiftedRevCountSeries = siftedDf.groupby("user_id").size()
    SiftedRevCountSeriesDict = SiftedRevCountSeries.to_dict()
    
    enthusiasmByCatDict = {k: SiftedRevCountSeriesDict[k]*SiftedAggStarMeanDict[k] for k in SiftedAggStarMeanDict}
    
    siftedListEnthusDicts.append((latentCat,enthusiasmByCatDict))
    
    print("Done "+latentCat)

Bars
Done Bars
Restaurants
Done Restaurants
Fashion
Done Fashion
Beauty & Spas
Done Beauty & Spas
Pubs
Done Pubs
Active Life
Done Active Life
Cosmetics & Beauty Supply
Done Cosmetics & Beauty Supply
Home & Garden
Done Home & Garden
Sports Clubs
Done Sports Clubs
Used
Done Used
Preschools
Done Preschools
Financial Services
Done Financial Services
Hair Removal
Done Hair Removal
Oil Change Stations
Done Oil Change Stations
Home Cleaning
Done Home Cleaning


In [117]:
testDict1 = {"one":7,"two":9,"three":11}
testDict2 = {"one":2,"two":3,"three":4}


In [118]:
{k: testDict1[k]*testDict2[k] for k in testDict1}

{'one': 14, 'two': 27, 'three': 44}

In [14]:
# READ IN THE GRAPH

In [2]:
#BigYelp = nx.read_edgelist("friendsFullNetworkEdges_PartialInfoAvailable.txt")

In [1]:
#try: 
    #nx.write_graphml(BigYelp,"friendsFullNetworkEdges_PartialInfoAvailable.graphml")
#except SerialisationError:
    #print(3)

In [2]:
yelpino = nx.read_edgelist("friendsSubNetworkEdges_InfoAvailable.txt")

In [5]:
nx.number_of_nodes(yelpino), yelpino.size()

(606508, 5322901)

In [7]:
BigYelp = nx.read_edgelist("friendsFullNetworkEdges_PartialInfoAvailable.txt")

In [8]:
nx.number_of_nodes(BigYelp), BigYelp.size()

(10678668, 44303860)

In [16]:
nx.set_node_attributes(yelpino, Yelp_ReviewsUserDict,"Reviews")

In [76]:
nx.set_node_attributes(yelpino, Yelp_ReviewsAggStarMeanDict, "AverageStar")

In [95]:
nx.set_node_attributes(yelpino, YelpReviewsRevCountSeriesDict, "ReviewCount")

In [122]:
for latentCatDictTuple in siftedListDicts:
    latentCategory = latentCatDictTuple[0]
    localDict = latentCatDictTuple[1]
    nx.set_node_attributes(yelpino, localDict, latentCategory)

In [115]:
for latCat in busLatCatsList:
    print(latCat + " assortativity: "+str(nx.attribute_assortativity_coefficient(yelpino,latCat)))

Bars assortativity: 0.02708162228366847
Restaurants assortativity: 0.0403651435345962
Fashion assortativity: 0.07108086867204101
Beauty & Spas assortativity: 0.08342029331960416
Pubs assortativity: 0.06724732150377787
Active Life assortativity: 0.09773029468230386
Cosmetics & Beauty Supply assortativity: 0.05674868615639566
Home & Garden assortativity: 0.09314802793194313
Sports Clubs assortativity: 0.07849327460664403
Used assortativity: 0.0690736806092073
Preschools assortativity: 0.0457145961510417
Financial Services assortativity: 0.0531528655192846
Hair Removal assortativity: 0.09022263489582312
Oil Change Stations assortativity: 0.04593194437626429
Home Cleaning assortativity: 0.06490892747692766


In [123]:
for latentCatDictEnthusTuple in siftedListEnthusDicts:
    latentCategory = latentCatDictEnthusTuple[0] + "_enthusiasm"
    print(latentCategory)
    localDict = latentCatDictEnthusTuple[1]
    nx.set_node_attributes(yelpino, localDict, latentCategory)

Bars_enthusiasm
Restaurants_enthusiasm
Fashion_enthusiasm
Beauty & Spas_enthusiasm
Pubs_enthusiasm
Active Life_enthusiasm
Cosmetics & Beauty Supply_enthusiasm
Home & Garden_enthusiasm
Sports Clubs_enthusiasm
Used_enthusiasm
Preschools_enthusiasm
Financial Services_enthusiasm
Hair Removal_enthusiasm
Oil Change Stations_enthusiasm
Home Cleaning_enthusiasm


In [124]:
for latCat in busLatCatsList:
    latCat = latCat+"_enthusiasm"
    print(latCat + " enthusiasm assortativity: "+str(nx.attribute_assortativity_coefficient(yelpino,latCat)))

Bars_enthusiasm enthusiasm assortativity: 0.02192287124656674
Restaurants_enthusiasm enthusiasm assortativity: 0.03627511539814643
Fashion_enthusiasm enthusiasm assortativity: 0.06788735870720426
Beauty & Spas_enthusiasm enthusiasm assortativity: 0.07471112694138975
Pubs_enthusiasm enthusiasm assortativity: 0.06441924516203301
Active Life_enthusiasm enthusiasm assortativity: 0.09242651536644597
Cosmetics & Beauty Supply_enthusiasm enthusiasm assortativity: 0.053342333502640206
Home & Garden_enthusiasm enthusiasm assortativity: 0.08746606430984903
Sports Clubs_enthusiasm enthusiasm assortativity: 0.07440785056301338
Used_enthusiasm enthusiasm assortativity: 0.06531539563115533
Preschools_enthusiasm enthusiasm assortativity: 0.04274641169593429
Financial Services_enthusiasm enthusiasm assortativity: 0.04954848219294601
Hair Removal_enthusiasm enthusiasm assortativity: 0.08188676823559288
Oil Change Stations_enthusiasm enthusiasm assortativity: 0.04361531503715075
Home Cleaning_enthusiasm

In [77]:
nx.attribute_assortativity_coefficient(yelpino,"AverageStar")

0.027079046453203907

In [96]:
nx.attribute_assortativity_coefficient(yelpino,"ReviewCount")

0.026268943031893734

In [8]:
smallDegreeDict = dict(yelpino.degree())

In [16]:
smallDegreeDict['--2HUmLkcNHZp0xw6AMBPg']

3

In [17]:
largeDegreeDict = dict(BigYelp.degree())

In [19]:
largeDegreeDict['--2HUmLkcNHZp0xw6AMBPg']

58

In [31]:
dfDegrees = pd.DataFrame.from_dict(smallDegreeDict,orient = 'index',columns = ['SmallYelp'])

In [47]:
dfDegrees['index1'] = dfDegrees.index

In [54]:
dfDegrees.head()

,SmallYelp,index1
oMy_rEb0UBEmMlu-zcxnoQ,2,oMy_rEb0UBEmMlu-zcxnoQ
cvVMmlU1ouS3I5fhutaryQ,18,cvVMmlU1ouS3I5fhutaryQ
nj6UZ8tdGo8YJ9lUMTVWNw,7,nj6UZ8tdGo8YJ9lUMTVWNw
JJ-aSuM4pCFPdkfoZ34q0Q,27,JJ-aSuM4pCFPdkfoZ34q0Q
HVUAmApa0fCbHHVJ0ALshw,29,HVUAmApa0fCbHHVJ0ALshw


In [55]:
dfDegrees['BigYelp'] = dfDegrees['index1'].apply(lambda ID: largeDegreeDict[ID])

In [59]:
dfDegrees['SmallYelp'].corr(dfDegrees['BigYelp'], method = 'pearson')

0.8685414501447632

In [58]:
dfDegrees['SmallYelp'].corr(dfDegrees['BigYelp'],method = 'spearman')

0.772002464623897

In [28]:
count1 = 0
count2 = 0
count3 = 0
count4 = 0
noNeighborInfo1 = 0
noNeighborInfo2 = 0
noNeighborInfo3 = 0
noNeighborInfo4 = 0

def neighborsAverageRating(userIdNode):
    global count1
    global noNeighborInfo1
    count1+=1
    userIdInQuestion = userIdNode
    totalAvStarSum = 0
    totalPeople = 0
    if count1 %2 == 0:
        print("1CountPerc: "+str(count1/10))
        # OVER 379868
    if userIdInQuestion in yelpino:
        listNeighbors = list(nx.all_neighbors(yelpino,userIdInQuestion))
        numNeighbors = len(listNeighbors)
        for neighborId in listNeighbors:
            try:
                testNodeAttributes = nx.get_node_attributes(yelpino,"Reviews")[neighborId] # get dataframe for neighbor
                if len(testNodeAttributes) > 0 :
                    sumOfStarsNeighbor = testNodeAttributes['stars'].sum()
                    sumOfNumReviews = len(testNodeAttributes['stars']) 
                    avStarNeighbor = sumOfStarsNeighbor/sumOfNumReviews 
                    totalAvStarSum+=avStarNeighbor
                    totalPeople+=1
            except KeyError:
                print("function 1, count = "+str(count1)+": user: "+userIdInQuestion +", friendId: "+ neighborId)
                noNeighborInfo1 +=1
                print("No Neighbor count: "+str(noNeighborInfo1))
        av = totalAvStarSum/totalPeople
        return (av,numNeighbors)
    else:
        return (np.NAN,numNeighbors)
    
def neighborsWeightedAverageRating(userIdNode):
    global count2
    global noNeighborInfo2
    count2+=1
    userIdInQuestion = userIdNode
    totalStarSum = 0
    totalReviews = 0
    if count2 %2 == 0:
        print("2CountPerc: "+str(count2/10))
        # OVER 379868
    if userIdInQuestion in yelpino:
        listNeighbors = list(nx.all_neighbors(yelpino,userIdInQuestion))
        numNeighbors = len(listNeighbors)
        for neighborId in listNeighbors:
            try:
                testNodeAttributes = nx.get_node_attributes(yelpino,"Reviews")[neighborId] # get dataframe for neighbor
                if len(testNodeAttributes) > 0 :
                    totalStarSum += testNodeAttributes['stars'].sum()
                    totalReviews += len(testNodeAttributes['stars']) 
            except KeyError:
                print("function 2, count = "+str(count2)+": user: "+userIdInQuestion +", friendId: "+ neighborId)
                noNeighborInfo2 +=1
                print("No Neighbor count: "+str(noNeighborInfo2))
        if totalReviews > 0:
            avRating = totalStarSum/totalReviews
            return (avRating,numNeighbors)
    else:
        return (np.NAN,numNeighbors)
    

def neighborsAverageRatingByBusinessCategory(tupleUserIdCat):
    global count3
    global noNeighborInfo3
    count3+=1
    userIdInQuestion = tupleUserIdCat[0]
    busLatCat = tupleUserIdCat[1]
    totalAvStarSum = 0
    totalPeople = 0
    if count3 %2 == 0:
        print("3CountPerc: "+str(count3/10))
        #770514
    if userIdInQuestion in yelpino:
        print("3: " + str(userIdInQuestion))
        listNeighbors = list(nx.all_neighbors(yelpino,userIdInQuestion))
        numNeighbors = len(listNeighbors)
        for neighborId in listNeighbors:
            try:
                testNodeAttributes = nx.get_node_attributes(yelpino,"Reviews")[neighborId] # get dataframe for neighbor
                if busLatCat in testNodeAttributes['businessLatentCategory'].values:
                    busCatSumOfStarsNeighbor = testNodeAttributes.groupby("businessLatentCategory").get_group(busLatCat)['stars'].agg("sum")
                    busCatSumOfNumReviews = len(testNodeAttributes.groupby("businessLatentCategory").get_group(busLatCat)['stars'])
                    busLatCatStarsMean = busCatSumOfStarsNeighbor/busCatSumOfNumReviews
                    totalAvStarSum+=busLatCatStarsMean
                    totalPeople+=1
            except KeyError:
                print("function 3, count = "+str(count3)+": user: "+userIdInQuestion +", friendId: "+ neighborId)
                noNeighborInfo3 +=1
                print("No Neighbor count: "+str(noNeighborInfo3))
        if totalPeople > 0:
            av = totalAvStarSum/totalPeople
            return (av,numNeighbors)
    else:
        return (np.NAN,numNeighbors)
    
def neighborsWeightedAverageRatingByBusinessCategory(tupleUserIdCat):
    global count4
    global noNeighborInfo4
    count4+=1
    userIdInQuestion = tupleUserIdCat[0]
    busLatCat = tupleUserIdCat[1]
    totalStarSum = 0
    totalReviews = 0
    if count4 %2 == 0:
        print("4CountPerc: "+str(count4/10))
        #OVER 770514
    if userIdInQuestion in yelpino:
        print("4: " + str(userIdInQuestion))
        listNeighbors = list(nx.all_neighbors(yelpino,userIdInQuestion))
        numNeighbors = len(listNeighbors)
        for neighborId in listNeighbors:
            try:
                testNodeAttributes = nx.get_node_attributes(yelpino,"Reviews")[neighborId] # get dataframe for neighbor
                if busLatCat in testNodeAttributes['businessLatentCategory'].values:
                    totalStarSum += testNodeAttributes.groupby("businessLatentCategory").get_group(busLatCat)['stars'].agg("sum")
                    totalReviews += len(testNodeAttributes.groupby("businessLatentCategory").get_group(busLatCat)['stars'])
            except KeyError:
                print("function 4, count = "+str(count4)+": user: "+userIdInQuestion +", friendId: "+ neighborId)
                noNeighborInfo4 +=1
                print("No Neighbor count: "+str(noNeighborInfo4))

        if totalReviews > 0:
            avRating = totalStarSum/totalReviews
            return (avRating,numNeighbors)
    else:
        return (np.NAN,numNeighbors) 

    








In [29]:
def testFunction(LIST): 
    List = LIST[0]
    whichCSV = LIST[2]
    if LIST[1] == 1: 
        csvfile = ''
        if whichCSV == 'first':
            csvfile = '1_first.csv'
        elif whichCSV == 'second':
            csvfile = '1_second.csv'
        with open(csvfile, 'a') as newFile:
            newFileWriter = csv.writer(newFile)
            for i in range(len(List)):
                locDic1 = {}
                locDic1[List[i]] = (neighborsAverageRating(List[i]))
                print(locDic1)
                newFileWriter.writerow([List[i], locDic1[List[i]]]) 
        return {}
    
    if LIST[1] == 2: 
        csvfile = ''
        if whichCSV == 'first':
            csvfile = '2_first.csv'
        elif whichCSV == 'second':
            csvfile = '2_second.csv'
        with open(csvfile, 'a') as newFile:
            newFileWriter = csv.writer(newFile)
            for i in range(len(List)):
                locDic2 = {}
                locDic2[List[i]] = (neighborsWeightedAverageRating(List[i]))
                print(locDic2)
                newFileWriter.writerow([List[i], locDic2[List[i]]]) 
        return {}
    
    if LIST[1] == 3: 
        csvfile = ''
        if whichCSV == 'first':
            csvfile = '3_first.csv'
        elif whichCSV == 'second':
            csvfile = '3_second.csv'
        with open(csvfile, 'a') as newFile:
            newFileWriter = csv.writer(newFile)
            for i in range(len(List)):
                locDic3 = {}
                locDic3[List[i]] = (neighborsAverageRatingByBusinessCategory(List[i]))
                print(locDic3)
                newFileWriter.writerow([List[i], locDic3[List[i]]]) 
        return {}
    
    if LIST[1] == 4: 
        csvfile = ''
        if whichCSV == 'first':
            csvfile = '4_first.csv'
        elif whichCSV == 'second':
            csvfile = '4_second.csv'
        with open(csvfile, 'a') as newFile:
            newFileWriter = csv.writer(newFile)
            for i in range(len(List)):
                locDic4 = {}
                locDic4[List[i]] = (neighborsWeightedAverageRatingByBusinessCategory(List[i]))
                print(locDic4)
                newFileWriter.writerow([List[i], locDic4[List[i]]]) 
        return {}


In [32]:
def f(arg):
    return testFunction(arg)

pool=Pool(processes=8)

listOfDicts = pool.map(f, [(REV_ListOfUSERS[0:379868],1,"first"),\
                                (REV_ListOfUSERS[379868:len(REV_ListOfUSERS)],1,"second"),\
                                (REV_ListOfUSERS[0:379868],2,"first"),\
                                (REV_ListOfUSERS[379868:len(REV_ListOfUSERS)],2,"second"),\
                                (REV_ListOfTuplesUSERS_BUSCAT[0:770514],3,"first"),\
                                (REV_ListOfTuplesUSERS_BUSCAT[770514:len(REV_ListOfTuplesUSERS_BUSCAT)],3,"second"),\
                                (REV_ListOfTuplesUSERS_BUSCAT[0:770514],4,"first"),\
                                (REV_ListOfTuplesUSERS_BUSCAT[770514:len(REV_ListOfTuplesUSERS_BUSCAT)],4,"second")])



3: ---1lKK3aKOuomHnwAkAow
4: ---1lKK3aKOuomHnwAkAow
4: ---1lKK3aKOuomHnwAkAow
3: ---1lKK3aKOuomHnwAkAow
{'--2HUmLkcNHZp0xw6AMBPg': (4.0, 3)}
2CountPerc: 0.2
{'--2HUmLkcNHZp0xw6AMBPg': (3.737037037037037, 3)}
1CountPerc: 0.2
{('---1lKK3aKOuomHnwAkAow', 'Used'): (4.067803030303032, 314)}
3CountPerc: 0.2
{('---1lKK3aKOuomHnwAkAow', 'Active Life'): (3.7985781990521326, 314)}
4CountPerc: 0.2
4: ---1lKK3aKOuomHnwAkAow
{'---1lKK3aKOuomHnwAkAow': (3.956697511281709, 314)}
1CountPerc: 0.2
{'---1lKK3aKOuomHnwAkAow': (3.7515268729641695, 314)}
2CountPerc: 0.2
{('---1lKK3aKOuomHnwAkAow', 'Used'): (3.8536585365853657, 314)}
4CountPerc: 0.2
{('---1lKK3aKOuomHnwAkAow', 'Active Life'): (3.874831507182268, 314)}
3CountPerc: 0.2
3: ---1lKK3aKOuomHnwAkAow
{('---1lKK3aKOuomHnwAkAow', 'Bars'): (3.7114461960246743, 314)}
4: ---1lKK3aKOuomHnwAkAow
{('---1lKK3aKOuomHnwAkAow', 'Bars'): (3.9382138883947047, 314)}
3: ---1lKK3aKOuomHnwAkAow
{('---1lKK3aKOuomHnwAkAow', 'Beauty & Spas'): (3.971264367816092, 314)}
4

Process ForkPoolWorker-67:
Process ForkPoolWorker-72:
Process ForkPoolWorker-66:
Process ForkPoolWorker-65:
Process ForkPoolWorker-68:


KeyboardInterrupt: 

Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-69:
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/condaenvs/jupyterhub/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/condaenvs/jupyterhub/lib/py